In [71]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
from sklearn import ensemble
from sklearn import metrics

fatal_police_shooting_us = pd.read_csv('/content/drive/My Drive/data/cleaned-data/cleaned-PoliceKillingsUS.csv', encoding="windows-1252")
demographic = pd.read_csv('/content/drive/My Drive/data/cleaned-data/demographic.csv', encoding="windows-1252")
data = pd.read_csv('/content/drive/My Drive/data/cleaned-data/combined.csv', encoding="windows-1252")

In [83]:
print('--fatal_police_shooting_us')
print('Length: {}'.format(len(fatal_police_shooting_us)))
print('--demographic')
print('Length: {}'.format(len(demographic)))
print('--data')
print('Before: {}'.format(len(data)))
data = data.dropna(axis=0)
print('After:  {}'.format(len(data)))

--fatal_police_shooting_us
Length: 2254
--demographic
Length: 29132
--data
Before: 1938
After:  1938


In [84]:
data.head()

,state,city,percent_completed_hs,poverty_rate,Median Income,share_white,share_black,share_native_american,share_asian,share_hispanic,id,name,date,manner_of_death,armed,age,gender,race,signs_of_mental_illness,threat_level,flee,body_camera
25,AK,Barrow,84.6,11.7,76902.0,16.9,1.0,61.2,9.1,3.1,1255.0,Vincent Nageak,10/02/16,shot,gun,36.0,M,N,False,attack,Not fleeing,False
32,AK,Big Lake,90.4,9.6,70988.0,86.1,0.2,7.0,0.5,3.1,2363.0,Jean R. Valescot,17/02/17,shot,gun,35.0,M,B,False,attack,Not fleeing,False
89,AK,Fairbanks,91.2,13.1,55229.0,66.1,9.0,10.0,3.6,9.0,836.0,Tristan Vent,08/09/15,shot,gun,19.0,M,N,False,attack,Not fleeing,False
90,AK,Fairbanks,91.2,13.1,55229.0,66.1,9.0,10.0,3.6,9.0,816.0,Vincent J. Perdue,09/09/15,shot,gun,33.0,M,N,False,attack,Car,False
91,AK,Fairbanks,91.2,13.1,55229.0,66.1,9.0,10.0,3.6,9.0,2618.0,Shawn Buck,25/05/17,shot,gun,23.0,M,W,False,attack,Car,False


In [91]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1938 entries, 25 to 29882
Data columns (total 22 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   state                    1938 non-null   object 
 1   city                     1938 non-null   object 
 2   percent_completed_hs     1938 non-null   float64
 3   poverty_rate             1938 non-null   float64
 4   Median Income            1938 non-null   float64
 5   share_white              1938 non-null   float64
 6   share_black              1938 non-null   float64
 7   share_native_american    1938 non-null   float64
 8   share_asian              1938 non-null   float64
 9   share_hispanic           1938 non-null   float64
 10  id                       1938 non-null   float64
 11  name                     1938 non-null   object 
 12  date                     1938 non-null   object 
 13  manner_of_death          1938 non-null   object 
 14  armed                 

In [87]:
from sklearn import preprocessing
from sklearn import model_selection
feature = ['percent_completed_hs','poverty_rate','Median Income','state','city',
           'share_white','share_black','share_asian','share_native_american',
           'share_hispanic','manner_of_death', 'armed', 'gender', 'race', 
           'city', 'state', 'signs_of_mental_illness', 'threat_level']
target = 'age'

X = data.dropna()[feature]
y = data[target]

object_cols = (X.dtypes == 'object')[X.dtypes == 'object'].index

X_train, X_test, y_train, y_test  = model_selection.train_test_split(X,y,train_size=0.8, test_size=0.2, random_state=0)

encoder = preprocessing.OneHotEncoder(handle_unknown='ignore', sparse=False)
X_OH_train_cols = pd.DataFrame(encoder.fit_transform(X_train[object_cols]))
X_OH_test_cols = pd.DataFrame(encoder.transform(X_test[object_cols]))

X_OH_train_cols.index = X_train.index
X_OH_test_cols.index = X_test.index

X_train = X_train.drop(object_cols, axis=1)
X_test = X_test.drop(object_cols, axis=1)

X_train = pd.concat([X_train, X_OH_train_cols], axis=1)
X_test = pd.concat([X_test, X_OH_test_cols], axis=1)


In [93]:
# predict age of victims

model = ensemble.RandomForestRegressor()
model = model.fit(X_train,y_train)

test_prediction = model.predict(X_test)
print('Random Forest Regressor MAE: {}'.format(metrics.mean_absolute_error(y_test, test_prediction)))

Random Forest Regressor MAE: 10.326045732805003


In [94]:
# predict age of victims

model = ensemble.GradientBoostingRegressor()
model = model.fit(X_train,y_train)

test_prediction = model.predict(X_test)
print('Gradient Boosting Regressor MAE: {}'.format(metrics.mean_absolute_error(y_test, test_prediction)))

Gradient Boosting Regressor MAE: 9.887069550335301


In [95]:
feature = ['percent_completed_hs','poverty_rate','Median Income','state','city',
           'share_white','share_black','share_asian','share_native_american',
           'share_hispanic','manner_of_death', 'armed', 'gender', 'age', 
           'city', 'state', 'signs_of_mental_illness', 'threat_level']
target = 'race'

X = data.dropna()[feature]
y = data[target]

object_cols = (X.dtypes == 'object')[X.dtypes == 'object'].index

X_train, X_test, y_train, y_test  = model_selection.train_test_split(X,y,train_size=0.8, test_size=0.2, random_state=0)

encoder = preprocessing.OneHotEncoder(handle_unknown='ignore', sparse=False)
X_OH_train_cols = pd.DataFrame(encoder.fit_transform(X_train[object_cols]))
X_OH_test_cols = pd.DataFrame(encoder.transform(X_test[object_cols]))

X_OH_train_cols.index = X_train.index
X_OH_test_cols.index = X_test.index

X_train = X_train.drop(object_cols, axis=1)
X_test = X_test.drop(object_cols, axis=1)

X_train = pd.concat([X_train, X_OH_train_cols], axis=1)
X_test = pd.concat([X_test, X_OH_test_cols], axis=1)

In [100]:
# predict age of victims
model = ensemble.RandomForestClassifier()
model = model.fit(X_train,y_train)

test_prediction = model.predict(X_test)
print('Random Forest Classifier Accuracy Score: {}'.format(metrics.accuracy_score(y_test, test_prediction)))
print(metrics.classification_report(y_test, test_prediction))

Random Forest Classifier Accuracy Score: 0.6005154639175257
              precision    recall  f1-score   support

           A       0.50      0.10      0.17        10
           B       0.57      0.53      0.55       102
           H       0.46      0.57      0.51        65
           N       0.00      0.00      0.00         8
           O       0.75      0.50      0.60         6
           W       0.67      0.70      0.68       197

    accuracy                           0.60       388
   macro avg       0.49      0.40      0.42       388
weighted avg       0.59      0.60      0.59       388



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [98]:
model.feature_importances_

array([0.02447458, 0.02138458, 0.02156719, ..., 0.01607287, 0.01550846,
       0.0044888 ])